In [117]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import norm

## Q1

In [7]:
df = pd.read_csv("input000.txt", index_col=0)
df = df[["Adj Close"]].pct_change().dropna()
df.columns = ["Daily Returns"]
df.head()

,Daily Returns
Date,
10/19/2021,0.015080
10/20/2021,0.003361
10/21/2021,0.001474
10/22/2021,-0.005285
10/25/2021,-0.000336


## Q2

In [21]:
df = pd.read_csv("input001.txt", index_col=0, parse_dates=True)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-10-18,143.449997,146.839996,143.160004,146.550003,145.733383,85589200
2021-10-19,147.009995,149.169998,146.550003,148.759995,147.931061,76378900
2021-10-20,148.699997,149.750000,148.119995,149.259995,148.428253,58418800
2021-10-21,148.809998,149.639999,147.869995,149.479996,148.647034,61421000
2021-10-22,149.690002,150.179993,148.639999,148.690002,147.861450,58883400
...,...,...,...,...,...,...
2022-10-11,139.899994,141.350006,138.220001,138.979996,138.979996,77033700
2022-10-12,139.130005,140.360001,138.160004,138.339996,138.339996,70433700
2022-10-13,134.990005,143.589996,134.369995,142.990005,142.990005,113224000


In [29]:
df.loc[datetime.datetime(
    year=2021, month=10, day=29), "Adj Close"]


148.965271

In [16]:
164.618805/148.965271

1.1050817676826166

In [38]:
df1 = df[["Adj Close"]].pct_change(freq="M", fill_method="ffill")
df1.columns = ["Monthly Returns"]
df1 = df1.dropna()
df1.iloc[0, 0] = df.loc[datetime.datetime(
    year=2021, month=11, day=30), "Adj Close"]/df.loc[datetime.datetime(
    year=2021, month=10, day=29), "Adj Close"] - 1
df1.iloc[5, 0] = df.loc[datetime.datetime(
    year=2022, month=5, day=31), "Adj Close"]/df.loc[datetime.datetime(
        year=2022, month=4, day=29), "Adj Close"] - 1
df1.iloc[-2, 0] = df.loc[datetime.datetime(
    year=2022, month=8, day=31), "Adj Close"]/df.loc[datetime.datetime(
        year=2022, month=7, day=29), "Adj Close"] - 1
df1

,Monthly Returns
Date,
2021-11-30,0.105082
2021-12-31,0.074229
2022-01-31,-0.015712
2022-02-28,-0.054066
2022-03-31,0.057473
2022-05-31,-0.054496
2022-06-30,-0.081430
2022-08-31,-0.031208
2022-09-30,-0.120977


## Q3

In [72]:
def q_3_solution(df):
    # You are provided the dataframe.
    #
    # 1. You may want to use regular expressions. Extract the 2 letter state code from the name_latest into a column 'State'
    #     and return the cleaned company name into a column 'long_name'.
    #     The state may identified in 'name_lastest' by finding the two letter code preceded by either "/" or "\" and
    #     optionally followed by "/" or "\".
    #     E.g. \NY\, /NY/, /NY, \NY    - are all variations on NY, New York.
    #     Note: Either of the letter may also be lowercase
    #     If you use regular expression we would like you to consider using the following pattern
    #     r'([\w\s\.\,]+)\s*[\\\/]{1}(\w{2})[\\\/]?' it will help you to extract the name and the state
    #     we suggest you to use the method extract with the regular expression.
    #     https://www.geeksforgeeks.org/python-pandas-series-str-extract/
    df['long_name'] = df["name_latest"].str.extract(
        pat=r'([\w\s\.\,]+)\s*[\\\/]{1}(\w{2})[\\\/]?')[0]
    df["State"] = df["name_latest"].str.extract(
        pat=r'([\w\s\.\,]+)\s*[\\\/]{1}(\w{2})[\\\/]?')[1].str.upper()
    
    #
    # 2. Where the name_latest has no state to extract, replace the NaN value in long_name with the name_latest value
    #     E.g. "Kentucky Bancshares Inc /KY/" -> "Kentucky Bancshares Inc", "KY"
    #          "Nicholas Financial Inc" -> "Nicholas Financial Inc", "NaN"
    #
    # Return the resulting dataframe with only the columns 'company_id', 'long_name' and 'State'
    # df.long_name.fillna(df.name_latest, inplace=True)
    df["long_name"] = df["long_name"].fillna(df["name_latest"])
    df = df[['company_id', 'long_name', 'State']].copy()
    # Your code here

    return df

In [90]:
df = pd.read_csv("input002.txt")
df = q_3_solution(df)
df

,company_id,long_name,State
0,1000045,Nicholas Financial Inc,NaN
1,1000180,Sandisk Corp,NaN
2,1000209,Medallion Financial Corp,NaN
3,1000228,Henry Schein Inc,NaN
4,1000229,Core Laboratories N V,NaN
...,...,...,...
12168,99313,Transnet Corp,NaN
12169,99359,Breeze-Eastern Corp,NaN
12170,99771,Trinity Capital Corp,NaN
12171,99780,Trinity Industries Inc,NaN


In [91]:
df["State"] = df["State"].str.upper()
df = df.groupby("State").count()
# df.columns = ["company_id_count"]
df

,company_id,long_name
State,,
BC,1,1
CA,20,20
CN,5,5
CO,3,3
CT,3,3
DC,1,1
DE,173,173
FA,1,1
FL,5,5


In [93]:
df["company_id"]

State
BC      1
CA     20
CN      5
CO      3
CT      3
DC      1
DE    173
FA      1
FL      5
FR      1
GA      3
GO      1
GR      1
HI      1
IL      1
IN      5
KS      1
KY      6
LA      1
LO      1
MA     11
MD     14
ME      3
MI      9
MN      7
MO      5
MS      5
NC      4
NE     24
NJ      8
NM      1
NV     10
NW      1
NY     10
OH     17
OK      1
OM      1
OR      2
PA     17
PR      1
RH      1
RI      3
RX      1
SC      5
SD      2
SH      2
SW      1
TB      1
TN      4
TX     12
UK      1
US      1
UT      2
VA     13
VT      1
WA      7
WI      4
WV      2
Name: company_id, dtype: int64

In [92]:
df1 = pd.DataFrame()
df1["State"] = df.index
df1["company_id_count"] = df["company_id"]
df1

,State,company_id_count
0,BC,NaN
1,CA,NaN
2,CN,NaN
3,CO,NaN
4,CT,NaN
5,DC,NaN
6,DE,NaN
7,FA,NaN
8,FL,NaN
9,FR,NaN


In [ ]:
def q_4_solution(df):
    # You are provided the solution to q_3 in a dataframe.
    # Please observe that the states are not always display the same way. example: you can have NY, ny, Ny
    # You should normalize the state 2 letters first. You should use capitalize all the letters.
    # Return a dataframe with the count of each company_id by state. Think about groupby.
    
    # Your code here
    
    return df
    
    
def q_5_solution(df_comp,df_indic):
    # You must have a solution to q_4 to solve this problem.
    # This problem is challenging - don't spend too much time here if you havn't solved the other problems first.
    # You are provided the solution to q_3 as a dataframe, df_comp.
    # You are also provided a dataframe, df_indic, which contains fundamentals for the companies in df_comp identified by 
    # the company_id.
    # 1. Extract all the ids from df_comp that are headquarted in the state with the highest count from your q_4_solution.
    #        (hint: this state might be the second smallest state of usa)
    # 2. Filter df_indic by these company ids. 
    # 3. It is highly suggested to merge both dataframes (we are recommending to use merge(df_comp, on='company_id'))
    # 4. Calculate the asset ('Assets') growth rate ('growth_rate') from 2014 - 2015 for each of the companies.
    #         (hint: df_indic['2015'] / df_indic['2014']) - 1
    # Return a dataframe containing only the 'company_id', 'long_name' and 'growth_rate' you just calculated
    
    # Your code here.
    
    return df
    
def q_6_solution(df):
    # You are provided the dataframe.
    # The data contains a list of options represented in a variation on OSI symbology
    # 1. Using regular expressions, split the symbol into its component parts 'UnderSym', 'ExDate', 'OpType', 'Strike'
    #     (hint: you may want to use the following pattern: '\.([A-Z]{1,6})(\d{6})([CP])([\d\.]+)' )
    # 2. Cast the expiration date to datetime and the strike column to float
    #     (hint: pd.to_datetime(df['ExDate']))
    # Return the resulting dataframe.
    
    # Your code here.
    
    return df

# Q5
q_5
12174

## Q6

In [ ]:
def q_6_solution(df):
    # You are provided the dataframe.
    # The data contains a list of options represented in a variation on OSI symbology
    # 1. Using regular expressions, split the symbol into its component parts 'UnderSym', 'ExDate', 'OpType', 'Strike'
    #     (hint: you may want to use the following pattern: '\.([A-Z]{1,6})(\d{6})([CP])([\d\.]+)' )
    # 2. Cast the expiration date to datetime and the strike column to float
    #     (hint: pd.to_datetime(df['ExDate']))
    # Return the resulting dataframe.

    # Your code here.

    return df


In [107]:
df = pd.read_csv("input005.txt")
df

,FeedSym
0,.COST221202C455
1,.EOG221021P152.2
2,.ORCL230120C55
3,.WB221202C7
4,.IWM221026C150
...,...
599172,.ROOT221028P3
599173,.FISV221021P76
599174,.FIS250117C60
599175,.RRC221028P23


In [108]:
df[['UnderSym', 'ExDate', 'OpType', 'Strike']] = df["FeedSym"].str.extract(
    r'\.([A-Z]{1,6})(\d{6})([CP])([\d\.]+)')
df["ExDate"] = pd.to_datetime(df['ExDate'])
df["Strike"] = df["Strike"].astype(float)
df

,FeedSym,UnderSym,ExDate,OpType,Strike
0,.COST221202C455,COST,2002-12-22,C,455.0
1,.EOG221021P152.2,EOG,2021-10-22,P,152.2
2,.ORCL230120C55,ORCL,2020-01-23,C,55.0
3,.WB221202C7,WB,2002-12-22,C,7.0
4,.IWM221026C150,IWM,2026-10-22,C,150.0
...,...,...,...,...,...
599172,.ROOT221028P3,ROOT,2028-10-22,P,3.0
599173,.FISV221021P76,FISV,2021-10-22,P,76.0
599174,.FIS250117C60,FIS,2017-01-25,C,60.0
599175,.RRC221028P23,RRC,2028-10-22,P,23.0


## Q7

In [113]:
data = df.copy()

In [116]:
df1 = data[data["ExDate"] > "2021-09-30"]
df1 = df1[(df1["Strike"] >= 150) & (df1["Strike"] <= 200)]
df1

,FeedSym,UnderSym,ExDate,OpType,Strike
1,.EOG221021P152.2,EOG,2021-10-22,P,152.2
4,.IWM221026C150,IWM,2026-10-22,C,150.0
22,.IWM221031P162,IWM,2031-10-22,P,162.0
146,.XSD221021P191,XSD,2021-10-22,P,191.0
184,.AXP221028P177.5,AXP,2028-10-22,P,177.5
...,...,...,...,...,...
598742,.GLD221028C151,GLD,2028-10-22,C,151.0
598941,.CAR221021P185,CAR,2021-10-22,P,185.0
598970,.CRM221125C170,CRM,2025-11-22,C,170.0
599126,.VTWG221021C169,VTWG,2021-10-22,C,169.0


In [ ]:
def q_7_solution(data):
    # Solve q_6 first.
    # Using the dataframe produced from q_6, provided to you.
    # Filter and return a dataframe containing all the options expiring after 9/30/21
    # and with strike prices between 150 and 200, inclusive
    # Return the dataframe
    data = data.loc[:]
    # Your code here.
    
    return df

## Q8
50

In [125]:
# QUESTION 8
# You will need to complete the call and put functions below.
# Please see the question description which gives you the black-scholes formula.
# These functions will be tested in q_8.
# They will also be used in q_9.
# Please round answers to 2 places.
N = norm.cdf


def bs_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + T * (r + sigma ** 2 / 2)) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = S * N(d1) - K * np.exp(-r * T) * N(d2)
    return call



def bs_put(S, K, T, r, sigma):
    d1 = (np.log(S / K) + T * (r + sigma ** 2 / 2)) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put = N(-d2) * K * np.exp(-r * T) - N(-d1) * S
    return put


In [119]:
N = norm.cdf
N(1)


0.8413447460685429

## Q9

In [123]:
data = df1.copy()
data

,FeedSym,UnderSym,ExDate,OpType,Strike
1,.EOG221021P152.2,EOG,2021-10-22,P,152.2
4,.IWM221026C150,IWM,2026-10-22,C,150.0
22,.IWM221031P162,IWM,2031-10-22,P,162.0
146,.XSD221021P191,XSD,2021-10-22,P,191.0
184,.AXP221028P177.5,AXP,2028-10-22,P,177.5
...,...,...,...,...,...
598742,.GLD221028C151,GLD,2028-10-22,C,151.0
598941,.CAR221021P185,CAR,2021-10-22,P,185.0
598970,.CRM221125C170,CRM,2025-11-22,C,170.0
599126,.VTWG221021C169,VTWG,2021-10-22,C,169.0


In [133]:
df = pd.DataFrame()
df["C"] = data.apply(lambda row: bs_call(160, row.Strike, (row.ExDate - datetime.datetime(2021, 10, 21)).days / 365, 0.02, 0.2), axis=1)
df["P"] = data.apply(lambda row: bs_put(160, row.Strike, (row.ExDate - datetime.datetime(2021, 10, 21)).days / 365, 0.02, 0.2), axis=1)
data['TheoPrice'] = df["C"]
data.loc[data['OpType'] == "P", 'TheoPrice'] = df["P"]
data

,FeedSym,UnderSym,ExDate,OpType,Strike,TheoPrice
1,.EOG221021P152.2,EOG,2021-10-22,P,152.2,2.785625e-07
4,.IWM221026C150,IWM,2026-10-22,C,150.0,4.003817e+01
22,.IWM221031P162,IWM,2031-10-22,P,162.0,2.415915e+01
146,.XSD221021P191,XSD,2021-10-22,P,191.0,3.098953e+01
184,.AXP221028P177.5,AXP,2028-10-22,P,177.5,3.002716e+01
...,...,...,...,...,...,...
598742,.GLD221028C151,GLD,2028-10-22,C,151.0,4.682419e+01
598941,.CAR221021P185,CAR,2021-10-22,P,185.0,2.498986e+01
598970,.CRM221125C170,CRM,2025-11-22,C,170.0,2.708672e+01
599126,.VTWG221021C169,VTWG,2021-10-22,C,169.0,2.729163e-08


In [ ]:
def q_9_solution(data):
    # You will likely need to solve q_8 first, but it is not required.
    # You are provided the solution to q_7 in a dataframe.
    # Assumptions:
    #     On 10/21/22, assume all stocks are trading at $160 and all options have a constant volatility of 20%.
    #     The annual interest rate is 2%.
    #     365 days in the year.
    # 1. Calculate the BlackScholes theoretical option price for the option and put it into 'TheoPrice' column.
    #    You will need to call the appropriate function bc_call or bs_put based on row['OpType'].
    #    We are suggesting to use the function apply to answer this question.
    #    We consider the keyword 'row' being one row on the dataframe data.
    #         T = (row['ExDate'] - datetime.datetime(2021, 10, 21)).days / 365
    # 2. Round price to 2 places.
    # Return the dataframe.
    S = 160
    r = 0.02
    sigma = 0.2

    data['T'] = (data['ExDate'] - datetime.datetime(2021, 10, 21)).days / 365

    # Your code here

    return df

In [1]:
import math
import os
import random
import re
import sys
import pandas as pd
import numpy as np
# from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_model import ARIMA

In [50]:
df = pd.read_csv("input011.txt", sep=",")
df["Month"] = pd.to_datetime(df["Month"])
df["Volume"] = df["Volume"].astype(int)
volume = df.copy()
volume.head()

,Month,Volume
0,1949-01-01,22400
1,1949-02-01,23600
2,1949-03-01,26400
3,1949-04-01,25800
4,1949-05-01,24200


In [44]:
np.log(volume.Volume)

0      10.016816
1      10.069002
2      10.181119
3      10.158130
4      10.094108
         ...    
139    11.705197
140    11.528799
141    11.431715
142    11.264464
143    11.366743
Name: Volume, Length: 144, dtype: float64

In [52]:
volume_log_diff = np.log(volume["Volume"]).diff()
volume_log_diff.to_frame("volume_log_diff")
volume_log_diff.index = volume["Month"]
volume_log_diff = volume_log_diff.dropna()

In [53]:
volume_log = np.log(volume["Volume"])
volume_log.to_frame("volume_log")
volume_log.index = volume["Month"]
volume_log

Month
1949-01-01    10.016816
1949-02-01    10.069002
1949-03-01    10.181119
1949-04-01    10.158130
1949-05-01    10.094108
                ...    
1960-08-01    11.705197
1960-09-01    11.528799
1960-10-01    11.431715
1960-11-01    11.264464
1960-12-01    11.366743
Name: Volume, Length: 144, dtype: float64

In [60]:
np.exp(volume_log.shift(1) + predictions_ARIMA_diff)

Month
1949-01-01              NaN
1949-02-01     22615.624507
1949-03-01     24016.428635
1949-04-01     27140.696741
1949-05-01     25683.624587
                  ...      
1960-08-01    119381.743103
1960-09-01    110508.481901
1960-10-01     92483.517824
1960-11-01     85973.271509
1960-12-01     77368.670061
Length: 144, dtype: float64

In [54]:
volume_log = np.log(volume["Volume"])
volume_log.to_frame("volume_log")
volume_log.index = volume["Month"]
# model = ARIMA(volume_log, order=(2, 1, 2))
# model.fit(disp=-1).summary()
model = ARIMA(volume_log, order=(2, 1, 2))
results_ARIMA = model.fit(disp=-1)
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_initial = np.exp(
    volume_log.iloc[0]+predictions_ARIMA_diff_cumsum)


/Users/zhangzixin/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/Users/zhangzixin/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


In [61]:
np.exp(volume_log.shift(1) + predictions_ARIMA_diff)

Month
1949-01-01              NaN
1949-02-01     22615.624507
1949-03-01     24016.428635
1949-04-01     27140.696741
1949-05-01     25683.624587
                  ...      
1960-08-01    119381.743103
1960-09-01    110508.481901
1960-10-01     92483.517824
1960-11-01     85973.271509
1960-12-01     77368.670061
Length: 144, dtype: float64

In [55]:
predictions_ARIMA_initial


Month
1949-02-01    22615.624507
1949-03-01    23014.683559
1949-04-01    23660.399511
1949-05-01    23553.675140
1949-06-01    22997.656495
                  ...     
1960-08-01    88377.341469
1960-09-01    80581.236305
1960-10-01    73350.750040
1960-11-01    68397.005950
1960-12-01    67843.402391
Length: 143, dtype: float64

In [28]:
predictions_ARIMA_initial

1      1.115044e+00
2      1.360312e+00
3      1.863089e+00
4      1.769767e+00
5      1.348917e+00
           ...     
139    5.965336e+06
140    2.088088e+06
141    7.172193e+05
142    3.239461e+05
143    2.953617e+05
Length: 143, dtype: float64

In [12]:
model = ARIMA(volume_log_diff, order=(2, 1, 2))
m = model.fit()
m

/Users/zhangzixin/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/zhangzixin/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:214: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/zhangzixin/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [62]:
import numpy as np
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

In [85]:
# Part 1
# Please do not modify this seed
np.random.seed(6996)

# error term
epsilon_vec = np.random.normal(0, 1, 500)  # to be completed
# X_matrix or regressors or predictiors
X_mat = np.random.normal(0, 2, (500, 500))  # to be completed
# Slope
slope_vec = np.random.uniform(1, 5, 500)  # to be completed
Y_mat = np.array(list(map(lambda z: list(
    1+np.sum(X_mat[:, :z]*slope_vec[:z], axis=1)+epsilon_vec), range(2, 501)))).T

In [114]:
np.dot(X_mat, slope_vec) + epsilon_vec


array([-9.51810407e+01,  2.89347410e+01, -1.83864664e+02, -1.64224386e+01,
       -4.36192011e+01,  2.47353183e+02, -1.15236391e+02,  1.72668277e+02,
       -1.19026091e+02, -2.45288187e+01,  4.15188362e+01, -5.00769986e+01,
        1.62521415e+02, -1.87230093e+02, -1.37566564e+02, -1.18212822e+02,
       -1.58358890e+02,  2.63101784e+02, -2.26768338e+02,  1.51875915e+02,
       -1.05703801e+02, -1.40511532e+02,  6.89830746e+01, -3.06573196e+02,
        8.99257398e+01, -3.41180589e+01, -2.49392075e+02, -2.12843904e+02,
       -1.92448801e+02,  3.88792377e+01, -1.22373908e+02,  1.80515412e+02,
        1.23195378e+02, -3.43173965e+01, -2.38923238e+01,  9.99048932e+01,
       -5.37036219e+01, -3.63129167e+01, -1.99223236e+02, -1.44876798e+02,
        2.96815319e+02,  1.79564068e+02,  1.50820411e+02, -2.81474351e+02,
        1.59211747e+01,  4.04642153e+01,  2.08374411e+02, -1.62717534e+02,
        4.56764262e+00,  1.51774855e+01,  2.00722190e+02, -3.40833840e+01,
        3.77066360e+01,  

In [112]:
y = np.sum(X_mat*slope_vec, axis=1) + epsilon_vec
regr = sm.OLS(Y_mat[:, 3], X_mat[:, :3]).fit()
regr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.497
Model:                            OLS   Adj. R-squared (uncentered):              0.494
Method:                 Least Squares   F-statistic:                              164.0
Date:                Sat, 05 Nov 2022   Prob (F-statistic):                    6.92e-74
Time:                        22:54:54   Log-Likelihood:                         -1844.4
No. Observations:                 500   AIC:                                      3695.
Df Residuals:                     497   BIC:                                      3707.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.5162      0.209      7.255      0.000       1.106       1.927
x2             1.4042      0.231      6.091      0.000       0.951       1.857
x3             4.2370      0.213     19.930      0.000       3.819       4.655
==============================================================================
Omnibus:                        4.405   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.111   Jarque-Bera (JB):                3.359
Skew:                          -0.068   Prob(JB):                        0.186
Kurtosis:                       2.622   Cond. No.                         1.14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [109]:
sm.regression()

array([1.33777515, 1.69535705, 4.28871913, 4.0323734 , 2.29845604,
       3.26999552, 3.32506172, 4.53238994, 1.50451277, 4.79443463,
       4.11469048, 2.96706836, 4.16447001, 2.01469321, 2.150951  ,
       3.83053391, 3.05777349, 2.87895657, 4.71948394, 1.32233633,
       4.08982743, 3.75837926, 3.24602213, 1.1738649 , 1.73064878,
       3.85502313, 1.75057879, 3.21850032, 1.59332278, 2.97925116,
       3.4572613 , 1.88953467, 3.89632355, 3.69353902, 1.31981295,
       3.44421281, 3.4698393 , 3.13593145, 3.35111741, 1.19931984,
       3.77046305, 4.63358987, 2.93289637, 2.38516299, 4.56923904,
       3.17121531, 1.70601799, 1.97236378, 2.65426569, 3.20963156,
       3.63885215, 3.24260817, 2.94807769, 3.17631997, 1.39710824,
       3.06046851, 1.71076152, 2.69344578, 3.00355316, 3.52942047,
       4.68336958, 2.86224333, 1.71361209, 4.66757663, 4.49267968,
       2.33092701, 1.37200275, 1.57739354, 3.48752822, 1.74222656,
       2.79739123, 2.19178695, 1.81821497, 2.18438995, 1.38643

In [100]:
regr = sm.OLS(Y_mat[:, 498], X_mat).fit()
regr.summary()

/Users/zhangzixin/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1698: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/zhangzixin/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1699: RuntimeWarning: invalid value encountered in double_scalars
  * (1 - self.rsquared))
/Users/zhangzixin/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1620: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 05 Nov 2022   Prob (F-statistic):                nan
Time:                        22:38:25   Log-Likelihood:                 12480.
No. Observations:                 500   AIC:                        -2.396e+04
Df Residuals:                       0   BIC:                        -2.185e+04
Df Model:                         499                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1421        inf          0        nan         nan         nan
x2             3.7236        inf          0        nan         nan         nan
x3             5.8325        inf          0        nan         nan         nan
x4             3.4087        inf          0        nan         nan         nan
x5             2.1960        inf          0        nan         nan         nan
x6             1.3228        inf          0        nan         nan         nan
x7             4.5819        inf          0        nan         nan         nan
x8             3.7960        inf          0        nan         nan         nan
x9             0.2407        inf          0        nan         nan         nan
x10            5.8226        inf          0        nan         nan         nan
x11            2.4592        inf          0        nan         nan         nan
x12            1.1054        inf          0        nan         nan         nan
x13            2.8475        inf          0        nan         nan         nan
x14            2.5025        inf          0        nan         nan         nan
x15            1.8136        inf          0        nan         nan         nan
x16            4.1920        inf          0        nan         nan         nan
x17            2.5713        inf          0        nan         nan         nan
x18            4.7644        inf          0        nan         nan         nan
x19            4.0896        inf          0        nan         nan         nan
x20            1.9387        inf          0        nan         nan         nan
x21            2.0589        inf          0        nan         nan         nan
x22            4.0493        inf          0        nan         nan         nan
x23            3.6582        inf          0        nan         nan         nan
x24            0.3101        inf          0        nan         nan         nan
x25            1.3003        inf          0        nan         nan         nan
x26            4.0470        inf          0        nan         nan         nan
x27            0.5082        inf          0        nan         nan         nan
x28            4.6389        inf          0        nan         nan         nan
x29            2.3913        inf          0        nan         nan         nan
x30            1.7448        inf          0        nan         nan         nan
x31            4.1838        inf          0        nan         nan         nan
x32            4.6350        inf          0        nan         nan         nan
x33            4.0397        inf          0        nan         nan         nan
x34            3.6964        inf          0        nan         nan         nan
x35            1.2832        inf          0        nan         nan         nan
x36            3.5132        inf          0        nan         nan         nan
x3

In [65]:
Y_mat = np.array(1+np.sum(X_mat[:, :z]*slope_vec[:z])+epsilon_vec for z in range(2, 500))
Y_mat

array(<generator object <genexpr> at 0x7fbf3cc9d3d0>, dtype=object)

In [74]:
np.sum(X_mat[:, :2]*slope_vec[:2], axis=1)

array([ 1.39585079e-01, -9.61132644e-01,  2.51153008e+00,  9.95909982e-01,
       -5.11273698e-03,  5.48500253e-01,  2.19261492e+00, -9.66072389e-01,
        9.65824356e-01, -8.99395073e-01, -1.26513975e+00, -3.08151454e+00,
       -4.66380539e+00,  4.32100188e-01, -5.28760340e+00,  1.72077795e+00,
        3.03641192e+00,  6.89870186e-01,  9.45177605e-01, -5.22200224e-01,
        2.84810622e+00,  9.26778830e-01, -1.54696554e-01,  1.23945969e+00,
        8.14391550e-01,  2.12173178e+00, -2.29591665e+00,  4.72498339e+00,
       -6.88858154e-01, -1.40141914e+00, -7.48949930e-01, -1.11596442e+00,
       -9.32500582e-03,  2.00871809e+00, -1.21820066e+00,  4.86327124e+00,
        2.20247123e-01, -2.31369710e+00, -1.19161079e+00, -4.67145448e+00,
       -5.66489405e-01, -2.69094195e+00,  1.09536218e+00,  3.36468726e-01,
       -3.58510437e-01,  6.03846168e-01, -4.77802595e+00, -2.78910802e-01,
        9.34715795e-01,  9.20109115e-01,  1.62188859e+00,  1.18328216e+00,
        8.90137056e-01,  

In [81]:
Y_mat = np.array(list(map(lambda z: list(
    1+np.sum(X_mat[:, :z]*slope_vec[:z], axis=1)+epsilon_vec), range(2, 501)))).T
Y_mat.shape

(500, 499)